In [9]:
# Imports

import car
from auto.capabilities import acquire, release, list_caps
from car.motors import set_throttle, set_steering
from auto.camera import global_camera, close_global_camera
from auto import console
import time

import numpy as np
import cv2
import os

list_caps()

('AHRS',
 'Accelerometer',
 'Buzzer',
 'Calibrator',
 'Camera',
 'CarMotors',
 'Credentials',
 'Encoders',
 'Gyroscope',
 'Gyroscope_accum',
 'LEDs',
 'LoopFrequency',
 'PID_steering',
 'PWMs',
 'Photoresistor',
 'Power',
 'PushButtons',
 'VersionInfo')

In [10]:
# Computer Vision constants
cv2.setNumThreads(0)

# BLUE = np.array([112,121, 189])
S_COLOR = np.array([100, 100, 125])
T_COLOR = np.array([225, 145, 130])

DIFF = 30

S_COLOR_LOWER_BOUND = np.clip(S_COLOR - DIFF, 0, 255)
S_COLOR_UPPER_BOUND = np.clip(S_COLOR + DIFF, 0, 255)

T_COLOR_LOWER_BOUND = np.clip(T_COLOR - DIFF, 0, 255)
T_COLOR_UPPER_BOUND = np.clip(T_COLOR + DIFF, 0, 255)

CROP_HEIGHT = 70
CROP_LEFT = 60
CROP_RIGHT = 60

In [11]:
# Functions

def crop(img):
    """
    Camera captures 240 pixels of height and 320 of width
    Shape is (240, 320, 3)
    """
    return img[-CROP_HEIGHT:, CROP_LEFT:-CROP_RIGHT]


def test_crop(img):
    return img[-120:, 0:320]


def bottom_portion(img):
    """
    Cut out the top half
    """
    return img[-120:, :]


def scale(img, scale_percent):
    # Calculate the new dimensions
    width = int(img.shape[1] * scale_percent)
    height = int(img.shape[0] * scale_percent)
    new_size = (width, height)

    # Resize the image:
    img = cv2.resize(img, new_size, None, None, None, cv2.INTER_AREA)
    return img


def percent_color(img, lower_bound, upper_bound, scale_percent=1):
    rgb_image = img
    rgb_mask = cv2.inRange(rgb_image, lower_bound, upper_bound)
    ratio = cv2.countNonZero(rgb_mask)/(img.size/3)
    color_percent = (ratio * 100) / scale_percent 
    return color_percent


def throttle_to_feet_per_second(throttle):
    return 0.33514 * throttle - 3.39442

def feet_per_second_to_throttle(feet_per_second):
    return (feet_per_second + 3.39442) / 0.33541

def throttle_to_steer_bias(throttle):
    return 0.68 * throttle -19.9


In [12]:
# Racing variables

# This is the only information needed to turn the corner
turn_radii = [3.75] # old is 3.75
turns = [-70]
num_turns = 1

NUM_LAPS = 1
MAX_THROTTLE = 22
MIN_THROTTLE = 18

STATIC_FRICTION = 0.7
GRAVITY = 9.81
WHEEL_BASE = 6.5 / 12 # 6.5 inches

In [13]:
class LoopParams:
    def __init__(prev_throttle, on_straight, gyroscope_handle, angle_accum, lap, frame, delta):
        
        self.prev_throttle = prev_throttle
        self.on_straight = on_straight
        self.gyroscope_handle = gyroscope_handle
        self.angle_accum = angle_accum
        self.lap = lap
        self.frame = frame
        self.delta = delta

In [16]:
# Custom logic
T_COLOR_THRESHOLD = 30.0   # % of StartTurnColor pixels to trigger a toggle, used to be 15%
T_COLOR_COOLDOWN  = 10.0    # seconds to ignore new StartTurnColors after a detection
THROTTLE_STEP   = 1
S_COLOR_THRESHOLD = 1.0 # As a percent
ANGLE_THRESHOLD = 15.0 # If greater than this on a straight and approaching a boundary reduce the throttle

state = "STRAIGHT"       # current driving mode
last_t_color_time = 0      # last time orange tape was seen
#orange: 37, 150, 190
turn_index = 0

def loop(prev_throttle, gyroscope, angle_accum, frame, delta):
    """
    Main control loop for Robocar F1:
    - Blue marks track boundaries
    - StartTurnColor marks the START/END of a turn
    """
    global state, last_t_color_time, turn_index, lap
    
    steer_bias = 0

    throttle = prev_throttle
    steer = 0
    steer_direction = 1

    t_color_img = bottom_portion(frame)
    blue_img  = test_crop(frame)
    t_color_percent = percent_color(t_color_img, T_COLOR_LOWER_BOUND, T_COLOR_UPPER_BOUND)
    z_angle = gyroscope.read()[2]
    now = time.time()

    if t_color_percent > T_COLOR_THRESHOLD and (now - last_t_color_time > T_COLOR_COOLDOWN):
        last_t_color_time = now

        # Toggle between straight and turn
        if state == "STRAIGHT":
            console.print("turn")
            state = "TURN"
        else:
            #console.print("straight");
            state = "STRAIGHT"
            # Update tracking for completed turn
            angle_accum += turns[turn_index]
            turn_index = (turn_index + 1) % num_turns
            if turn_index == 0:
                lap += 1
                
    if state == "STRAIGHT":
        steer_bias = -6.0
        
        z = gyroscope.read()[2]
        if z - angle_accum > 0:
            steer_direction = -1

        steer = np.abs(z) / 2
        
        blue_img = crop(frame)
        blue_percent = percent_color(blue_img, S_COLOR_LOWER_BOUND, S_COLOR_UPPER_BOUND)
        if blue_percent > S_COLOR_THRESHOLD and steer > ANGLE_THRESHOLD:
            throttle -= 10
        else:
            throttle += 1

        throttle = min(max(MIN_THROTTLE, throttle), MAX_THROTTLE) # Clip it on either side

    # Turns
    elif state == "TURN":
        """
        Due to error and good racing doesn't follow the perfect line, this can be deviated from, but anything widely
        off of this angle will be very very inefficient and maybe even dangerous
        """
        steer_bias = 0
        
        turn_radius = turn_radii[turn_index]
        steer_direction = -1 if turns[turn_index] > 0 else 1

        # Convert to deg
        steer = np.arctan(WHEEL_BASE / turn_radius) * 180 / np.pi
        
        steer = 0.597753 * steer - 2.22879
        
        # Would be nice if we could two pieces of color tape on the corner, one blue and one another color (not StartTurnColor)

        throttle = feet_per_second_to_throttle (np.sqrt(turn_radius * STATIC_FRICTION * GRAVITY))
    
#         steer_bias = -6.0
        
#         # Turn at fixed throttle, using radius for steering
#         turn_radius = turn_radii[turn_index]
#         steer_direction = -1 if turns[turn_index] > 0 else 1
#         throttle = 45

#         # Compute turning angle based on velocity and curve geometry
#         steer = throttle_to_feet_per_second(throttle) * delta / turn_radius
#         steer *= 180 / np.pi
#         steer = (steer * steer_direction) + 16

    throttle = np.clip(throttle, MIN_THROTTLE, MAX_THROTTLE)
    return (steer * steer_direction) + steer_bias, throttle, angle_accum


# on_straight = False
# StartTurnColor_WAIT = 1.0 # Can't begin detecting for a corner start/end until we wait this long
# last_StartTurnColor_detection = 0.0

# turn_index = 0 # Index into turns array
# lap = 0

# STRAIGHT_STEER_BIAS = -6.0 # The measured offset that the car is from straight (this is a motor problem, the gyroscope understands that it's 6 degrees off)

# BLUE_THRESHOLD = 1.0 # As a percent
# StartTurnColor_THRESHOLD = 100.0 # As a percent
# ANGLE_THRESHOLD = 15.0 # If greater than this on a straight and approaching a boundary reduce the throttle

# def start():
#     return 0, MAX_THROTTLE


# def loop(prev_throttle, on_straight, gyroscope, angle_accum, frame, delta):
#     global last_StartTurnColor_detection
#     global turn_index
#     steer = 0
#     steer_direction = 1
    
#     throttle = prev_throttle
    
#     if time.time() - last_StartTurnColor_detection >= StartTurnColor_WAIT:
#         StartTurnColor_img = bottom_portion(frame)
#         StartTurnColor_percent = percent_color(StartTurnColor_img, StartTurnColor_LOWER_BOUND, StartTurnColor_UPPER_BOUND)
#         if StartTurnColor_percent > StartTurnColor_THRESHOLD:
#             on_straight = not on_straight
#             last_StartTurnColor_detection = time.time()

#             if on_straight:
#                 angle_accum += turns[turn_index] # We have to do it like this in case the car isn't straight coming out of the corner
#                 turn_index += 1
#                 if turn_index >= num_turns:
#                     turn_index = 0
#                     lap += 1

#                 throttle = MAX_THROTTLE
        

    # For Straight
#     if state == "STRAIGHT":
#         steer_bias = -6.0
        
#         # Smooth acceleration
#         throttle = min(throttle + THROTTLE_STEP, MAX_THROTTLE)
        
#         # Center between blue boundaries
#         blue_mask = cv2.inRange(blue_img, S_COLOR_LOWER_BOUND, S_COLOR_UPPER_BOUND)
#         M = cv2.moments(blue_mask)
#         if M["m00"] != 0:
#             cx = int(M["m10"] / M["m00"])
#             center_offset = cx - (blue_img.shape[1] // 2)
#             steer = center_offset / 20.0

#         # Slow slightly if drifting too far
#         if abs(z_angle - angle_accum) > ANGLE_THRESHOLD:
#             throttle = max(throttle - 5, MIN_THROTTLE)
                
#     else:
#         """
#         Formula for theoretical turn angle is (v * dt / r)
#         Due to error and good racing doesn't follow the perfect line, this can be deviated from, but anything widely
#         off of this angle will be very very inefficient and maybe even dangerous
#         """
#         turn_radius = turn_radii[turn_index]
#         steer_direction = -1 if turns[turn_index] > 0 else 1
    
#         throttle = 60

#         # Would be nice if we could two pieces of color tape on the corner, one blue and one another color (not StartTurnColor)

#         steer = throttle_to_feet_per_second(throttle) * delta / turn_radius
#         # Steer is in radians, convert to deg
#         steer *= 180 / np.pi
#         return (steer * steer_direction) + 16, throttle, on_straight, angle_accum
     

In [17]:
console.clear()

gyroscope = acquire('Gyroscope_accum')
gyroscope.read()

camera = global_camera(False)
stream = camera.stream() # Returns a generator
next(stream)

throttle = MIN_THROTTLE

angle_accum = 0
dt = 0.25
lap = 0
while lap < NUM_LAPS:
    if stream:
        start = time.time()
        steer, throttle, angle_accum = loop(throttle, gyroscope, angle_accum, next(stream), dt)
        set_throttle(int(throttle))  # convert to native int
        set_steering(steer)
        dt = time.time() - start
        

release(gyroscope)
close_global_camera(False)

KeyboardInterrupt: 

In [18]:
while True:
    frame = car.capture()
    bottom = bottom_portion(frame)
    console.print(percent_color(bottom, StartTurnColor_LOWER_BOUND, StartTurnColor_UPPER_BOUND))


Captured 1 frame.


NameError: name 'StartTurnColor_LOWER_BOUND' is not defined

In [ ]:
while True:
    set_steering(40)

In [47]:
console.clear()

IndentationError: expected an indented block (<ipython-input-1-e546faa82dc9>, line 75)